<a href="https://colab.research.google.com/github/alexlopespereira/mba_enap/blob/main/CD/Atividades_Exercicios/Aula3_Exercicio3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Exercício 3.1 

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')


Authenticated


In [ ]:
import pandas as pd
## Defina o id do seu projeto no bigquery!!!!!
project_id = 'SEU PROJECT ID' # Defina o id do seu projeto no bigquery!!!!!
## Defina o id do seu projeto no bigquery!!!!!

df = pd.io.gbq.read_gbq('''
SELECT ... ''', project_id=project_id)

df.head()

,ano,sigla_uf,id_municipio,populacao,nome_municipio,pib,pibpercapita
0,2002,RO,1100015,27237.0,Alta Floresta D'oeste,111290995,4086.022506
1,2003,RO,1100015,27563.0,Alta Floresta D'oeste,143222381,5196.182600
2,2004,RO,1100015,29001.0,Alta Floresta D'oeste,173990790,5999.475535
3,2005,RO,1100015,28629.0,Alta Floresta D'oeste,167127355,5837.694471
4,2006,RO,1100015,29005.0,Alta Floresta D'oeste,168805060,5819.860714


In [ ]:
## Explique o que fez na sua query e por que.
# Fiz o left join da população porque essa tabela tem mais anos.
# Tive que fazer o distinct da tabela geobr_mapas porque essa tabela granularidade de setor censitário.

## Submeta o seu dataframe criando uma tabela no BigQuery

In [ ]:
df.to_gbq("enapdatasets.pibpercapita",
              project_id=project_id,
              chunksize=40000,
              if_exists='replace',
              )

5it [00:23,  4.79s/it]


#### O comando acima cria um dataset chamado enapdatasets e dentro dele uma tabela chamada pibpercapita.
#### Por padrão, os dados são guardados fisicamente nos EUA, onde também estão guardados os dados do projeto basedosdados.
#### Não é possível fazer join de dados guardados em locais distintos

##### Aqui há uma demonstração de como [criar um dataset manualmente](https://youtu.be/h_HRXdket7U)